In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras as keras
import matplotlib.pyplot as plt

In [2]:
def load_data(data_path):

    with open(data_path, 'r') as fp:
        data = json.load(fp)

    X = np.array(data['mfcc'])
    y = np.array(data['species'])

    return X, y

In [3]:
def prepare_datasets(test_size, validation_size):
    
    X, y = load_data(dataset_path)

    label_encoder = LabelEncoder()
    label_encoder.fit(y)
    new_y = label_encoder.transform(y)
    new_y = new_y.reshape(-1, 1)

    X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size = test_size)

    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size = validation_size)
    
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test

In [4]:
def build_model(input_shape):
    # sequential model
    model = keras.Sequential()
    # input layer
    model.add(keras.layers.Conv2D(32, (2, 2), input_shape=input_shape, activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides = (2, 2), padding = 'same'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.2))
    # second convolutional layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides = (2, 2), padding = 'same'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.2))
    # third convolutional layer
    model.add(keras.layers.Conv2D(64, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides = (2, 2), padding = 'same'))
    model.add(keras.layers.BatchNormalization())
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.GlobalAveragePooling2D())
    # flatten/dense layer    
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation = 'relu'))
    model.add(keras.layers.Dropout(0.2))
    # output layer
    model.add(keras.layers.Dense(49, activation='softmax'))

    return model

In [5]:
def plot_history(history):
    
    fig, axis = plt.subplots(2)
    axis[0].plot(history.history['accuracy'], label = 'train accuracy')
    axis[0].plot(history.history['val_accuracy'], label = 'validation accuracy')
    axis[0].set_ylabel("Accuracy")
    axis[0].legend(loc = 'lower right')
    axis[0].set_title('Accuracy Eval')
    
    axis[1].plot(history.history['loss'], label = 'train error')
    axis[1].plot(history.history['val_loss'], label = 'validation error')
    axis[1].set_ylabel("Error")
    axis[1].set_ylabel('Epoch')
    axis[1].legend(loc = 'upper right')
    axis[1].set_title('Error Eval')
    
    plt.show()

In [6]:
def predict(model, X, y):

    X = X[np.newaxis, ...]
    prediction = model.predict(X)
    predicted_index = np.argmax(prediction, axis = 1)
    print('Expected index: {}, Predicted index:{}'.format(y, predicted_index))

In [7]:
dataset_path = 'c:/users/ktauf/documents/github/finalproject_data_files/data_5segments_40mfccs.json'


In [ ]:
if __name__ == '__main__':

    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

    input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
    model = build_model(input_shape)

    optimizer = keras.optimizers.Adam(learning_rate = 0.0001)
    model.compile(optimizer = optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

    history = model.fit(X_train, y_train, validation_data = (X_validation, y_validation),
                        batch_size = 32, epochs = 25, verbose = 2)
    
    plot_history(history)
    
    model.summary()
    

In [9]:
test_error, test_accuracy = model.evaluate(X_test, y_test, verbose = 2)




2629/2629 - 4s - loss: 2.4202 - accuracy: 0.3830


In [11]:
X = X_test[300]
y = y_test[300]
predict(model, X, y)

Expected index: [10], Predicted index:[18]


In [ ]:
# save model
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

# save weights
model.save_weights('model.h5')